In [151]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, hog
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import os
import json
from scipy.spatial.distance import euclidean

In [152]:
def extract_color_histogram(frame, bins=64):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv_hist = cv2.calcHist([hsv], [0, 1, 2], None, [bins]*3, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hsv_hist, hsv_hist)
    return hsv_hist.flatten()

In [153]:
def extract_brightness_contrast_features(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return np.mean(gray), np.std(gray)


In [154]:
def extract_lbp_features(frame, radius=3, n_points=24, method='uniform'):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, n_points, radius, method)
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

In [155]:
def extract_hog_features(frame, resize_dim=(128, 128)):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_resized = cv2.resize(gray, resize_dim)
    hog_features = hog(gray_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
    return hog_features

In [156]:
def extract_sift_features(frame, n_clusters=100):  # Tăng số cụm SIFT lên 100
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is not None and len(descriptors) >= n_clusters:
        kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=0)
        kmeans.fit(descriptors)
        return kmeans.cluster_centers_.flatten()
    return np.zeros(n_clusters * 128)

In [157]:
def process_video(video_path, sample_rate=10, resize_dim=(128, 128)):  # Tăng tần suất lấy mẫu lên
    cap = cv2.VideoCapture(video_path)
    color_features, brightness_features, contrast_features, lbp_features, hog_features, sift_features = [], [], [], [], [], []
    frame_index = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index % sample_rate == 0:
            color_features.append(extract_color_histogram(frame))
            brightness, contrast = extract_brightness_contrast_features(frame)
            brightness_features.append(brightness)
            contrast_features.append(contrast)
            lbp_features.append(extract_lbp_features(frame))
            hog_features.append(extract_hog_features(frame, resize_dim))
            sift_features.append(extract_sift_features(frame))
        frame_index += 1
    cap.release()

    avg_color = np.mean(color_features, axis=0)
    avg_brightness = np.mean(brightness_features)
    avg_contrast = np.mean(contrast_features)
    avg_lbp = np.mean(lbp_features, axis=0)
    avg_hog = np.mean(hog_features, axis=0)
    avg_sift = np.mean(sift_features, axis=0)

    return avg_color, avg_brightness, avg_contrast, avg_lbp, avg_hog, avg_sift


In [158]:
def apply_pca(features, n_components=100):
    if features.shape[0] <= 1:
        return features  # Không đủ mẫu để áp dụng PCA
    if features.shape[1] < n_components:
        n_components = features.shape[1]
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    return reduced_features

In [159]:
def process_all_videos(directory_path, sample_rate=10, resize_dim=(128, 128), pca_components=100):
    data_directory = 'data'
    os.makedirs(data_directory, exist_ok=True)
    features_database = {}

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.mp4'):
            video_path = os.path.join(directory_path, file_name)
            avg_color, avg_brightness, avg_contrast, avg_lbp, avg_hog, avg_sift = process_video(video_path, sample_rate, resize_dim)

            # Ensure avg_sift has sufficient features for PCA
            avg_sift = avg_sift.reshape(1, -1)
            if avg_sift.shape[1] > 1:
                avg_sift = apply_pca(avg_sift, pca_components).flatten()  # Reduce SIFT dimensions

            features_database[file_name] = {
                'avg_color': avg_color.tolist(),
                'avg_brightness': avg_brightness,
                'avg_contrast': avg_contrast,
                'avg_lbp': avg_lbp.tolist(),
                'avg_hog': avg_hog.tolist(),
                'avg_sift': avg_sift.tolist()
            }

            with open(os.path.join(data_directory, f'{file_name}_features.json'), 'w') as f:
                json.dump(features_database[file_name], f, indent=4)

            print(f"Processed and saved features for video {file_name} in JSON format")

    return features_database


In [160]:
directory_path = '/kaggle/input/data-2/data-2'
process_all_videos(directory_path, sample_rate=10, resize_dim=(128, 128), pca_components=100)

Processed and saved features for video 019.mp4 in JSON format
Processed and saved features for video 017.mp4 in JSON format
Processed and saved features for video 026.mp4 in JSON format
Processed and saved features for video 030.mp4 in JSON format
Processed and saved features for video 009.mp4 in JSON format
Processed and saved features for video 028.mp4 in JSON format
Processed and saved features for video 015.mp4 in JSON format
Processed and saved features for video 013.mp4 in JSON format
Processed and saved features for video 012.mp4 in JSON format
Processed and saved features for video 004.mp4 in JSON format
Processed and saved features for video 003.mp4 in JSON format
Processed and saved features for video 020.mp4 in JSON format
Processed and saved features for video 025.mp4 in JSON format
Processed and saved features for video 002.mp4 in JSON format
Processed and saved features for video 018.mp4 in JSON format
Processed and saved features for video 008.mp4 in JSON format
Processe

{'019.mp4': {'avg_color': [1.5290144801838323e-05,
   1.536609124741517e-05,
   0.0003381621208973229,
   0.00033722742227837443,
   0.0006711503956466913,
   0.0006004678434692323,
   0.0026772129349410534,
   0.0027757694479078054,
   0.0023681577295064926,
   0.002652384340763092,
   0.0023907783906906843,
   0.0037703600246459246,
   0.003887702012434602,
   0.011322177946567535,
   0.020018553361296654,
   0.029486609622836113,
   0.05472738668322563,
   0.05744708329439163,
   0.08496509492397308,
   0.05382861942052841,
   0.06871975213289261,
   0.032044488936662674,
   0.022286072373390198,
   0.011960224248468876,
   0.008338979445397854,
   0.01127079501748085,
   0.014504995197057724,
   0.021502520889043808,
   0.010189706459641457,
   0.008073999546468258,
   0.0029257710557430983,
   0.0025090023409575224,
   0.0036765295080840588,
   0.0026627276092767715,
   0.003185571637004614,
   0.002256637206301093,
   0.0028706728480756283,
   0.002015229081735015,
   0.001513427